# NFL Teams Data Ingestion
This notebook loads NFL team information from nflreadr package to ADLS.

**USAGE**: This notebook is designed to be run on-demand only, not scheduled.
Run this job only when:
1. Initial data load
2. NFL team relocations (e.g., Raiders to Las Vegas)
3. Team rebrands (e.g., Washington Football Team to Commanders)
4. League expansion (extremely rare)

In [ ]:
# Install and load R packages
print("Installing and loading R packages...")
if(!require(arrow)) {
    install.packages("arrow", repos="https://cloud.r-project.org", quiet=TRUE)
}
if(!require(nflreadr)) {
    install.packages("nflreadr", repos="https://cloud.r-project.org", quiet=TRUE)
}
if(!require(tidyverse)) {
    install.packages("tidyverse", repos="https://cloud.r-project.org", quiet=TRUE)
}

library(nflreadr)
library(tidyverse)
library(arrow)

In [ ]:
# Get ADLS configuration from environment variables
container <- Sys.getenv("DFL_CONTAINER")
storage_account <- Sys.getenv("DFL_STORAGE_ACCOUNT")

# Build ADLS path
adls_path <- sprintf("abfss://%s@%s.dfs.core.windows.net/raw/nflverse/teams",
                    container,
                    storage_account)

# Load and save teams data with error handling
tryCatch({
    teams_data <- nflreadr::load_teams()
    output_file <- file.path(adls_path, "nfl_teams.parquet")
    print(paste("Attempting to save to:", output_file))
    
    # Write directly to ADLS using arrow with ABFSS
    write_parquet(teams_data, output_file)
    print("File successfully written to ADLS")
    
}, error = function(e) {
    print(paste("Error:", e$message))
    stop(e)  # Re-throw the error to ensure the job fails if there's an issue
})